# Trade Manager

# Buy agent

In [ ]:
from agents.buy_agent_trainer import BuyAgentTrainer
from agents.sell_agent_trainer import SellAgentTrainer

buy_trainer = BuyAgentTrainer(
    ticker="AAPL",
    window_size=30,
    horizon=20,
    transaction_cost=0.001,
    lambda_dd=0.05,
    lambda_vol=0.01,
    hold_penalty_long=0.0,
    device="cpu",
)

buy_history = buy_trainer.train(
    n_episodes=50,
    warmup_dynamic=True,
    verbose=True,
)


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
[*********************100%***********************]  1 of 1 completed


[BuyTrainer] Raw dataset: (1224, 10)
[BuyTrainer] After dropna: (1224, 10)
[BuyTrainer] Rolling state_df shape: (1194, 270)
[BuyTrainer] state_dim=270, actions=2
[BuyTrainer] Dynamic warmup set to: 238
[Episode 1/50] Reward=0.3147 | MeanTrades=0.0070 | Eps=0.887 | Steps=1193 | Trades=45 | Buffer=1193 | Avg10=0.3147
[Episode 5/50] Reward=0.2105 | MeanTrades=0.0047 | Eps=0.433 | Steps=1193 | Trades=45 | Buffer=5965 | Avg10=0.4676
[Episode 10/50] Reward=0.1486 | MeanTrades=0.0037 | Eps=0.050 | Steps=1193 | Trades=40 | Buffer=11930 | Avg10=0.3552
[Episode 15/50] Reward=0.8257 | MeanTrades=0.0201 | Eps=0.050 | Steps=1193 | Trades=41 | Buffer=17895 | Avg10=0.4519
[Episode 20/50] Reward=0.2113 | MeanTrades=0.0057 | Eps=0.050 | Steps=1193 | Trades=37 | Buffer=23860 | Avg10=0.4420
[Episode 25/50] Reward=0.6200 | MeanTrades=0.0151 | Eps=0.050 | Steps=1193 | Trades=41 | Buffer=29825 | Avg10=0.3511
[Episode 30/50] Reward=1.0144 | MeanTrades=0.0254 | Eps=0.050 | Steps=1193 | Trades=40 | Buffer=3579

# Sell Agent

In [2]:
sell_trainer = SellAgentTrainer(
    ticker="AAPL",
    window_size=30,
    horizon=20,
    transaction_cost=0.001,
    lambda_dd=0.05,
    lambda_vol=0.01,
    hold_penalty_long=0.0,
    device="cpu",
)

sell_history = sell_trainer.train(
    n_episodes=50,
    warmup_dynamic=True,
    verbose=True,
)

[*********************100%***********************]  1 of 1 completed

[SellTrainer] Raw dataset: (1224, 10)
[SellTrainer] After dropna: (1224, 10)
[SellTrainer] state_df shape: (1194, 270)
[SellTrainer] state_dim=270, actions=2
[SellTrainer] Dynamic warmup set to: 238
[Sell Ep 1/50] Reward=0.0215 | MeanTrade=0.0215 | Eps=0.998 | Steps=4 | Buffer=4 | Avg10=0.0215
[Sell Ep 5/50] Reward=-0.0709 | MeanTrade=-0.0709 | Eps=0.986 | Steps=6 | Buffer=29 | Avg10=-0.0188
[Sell Ep 10/50] Reward=0.0151 | MeanTrade=0.0151 | Eps=0.974 | Steps=7 | Buffer=54 | Avg10=0.0022
[Sell Ep 15/50] Reward=-0.0008 | MeanTrade=-0.0008 | Eps=0.960 | Steps=4 | Buffer=84 | Avg10=0.0086
[Sell Ep 20/50] Reward=-0.1001 | MeanTrade=-0.1001 | Eps=0.948 | Steps=4 | Buffer=109 | Avg10=-0.0111
[Sell Ep 25/50] Reward=0.0122 | MeanTrade=0.0122 | Eps=0.937 | Steps=4 | Buffer=132 | Avg10=-0.0187
[Sell Ep 30/50] Reward=-0.0528 | MeanTrade=-0.0528 | Eps=0.925 | Steps=4 | Buffer=157 | Avg10=-0.0392
[Sell Ep 35/50] Reward=0.0219 | MeanTrade=0.0219 | Eps=0.915 | Steps=6 | Buffer=179 | Avg10=-0.0331
[Se

In [ ]:
from scripts.trade_manager import TradeManager
from agents.buy_agent_trainer import BuyAgentTrainer
from agents.sell_agent_trainer import SellAgentTrainer

# Assuming you've already done:
# buy_trainer = BuyAgentTrainer(...); buy_trainer.train(...)
# sell_trainer = SellAgentTrainer(...); sell_trainer.train(...)

tm = TradeManager(
    buy_trainer=buy_trainer,
    sell_trainer=sell_trainer,
    cfg_path="config/data_config.yaml",
)

results = tm.run_backtest(greedy=True)

equity = results["equity_curve"]
trades = results["trades"]

print("Trades executed:", len(trades))
print("First 5 trades:")
for t in trades[:5]:
    print(t)

print("Final equity:", equity[-1])


In [4]:
equity = results["equity_curve"]
trades = results["trades"]

print("Trades executed:", len(trades))
print("First 5 trades:")
for t in trades[:5]:
    print(t)

print("Final equity:", equity[-1])


Trades executed: 317
First 5 trades:
{'entry_idx': 0, 'exit_idx': 1, 'entry_price': 46.71654510498047, 'exit_price': 47.0292854309082, 'return': 0.006694423254651017}
{'entry_idx': 2, 'exit_idx': 3, 'entry_price': 47.76933288574219, 'exit_price': 47.6260986328125, 'return': -0.002998456211902405}
{'entry_idx': 4, 'exit_idx': 5, 'entry_price': 47.89347457885742, 'exit_price': 47.49478530883789, 'return': -0.0083245008537245}
{'entry_idx': 6, 'exit_idx': 7, 'entry_price': 47.4757080078125, 'exit_price': 47.561641693115234, 'return': 0.0018100558982415452}
{'entry_idx': 8, 'exit_idx': 9, 'entry_price': 47.56641387939453, 'exit_price': 48.49267578125, 'return': 0.019473023638149848}
Final equity: 1.8445195601257527
